In [1]:
import chainlit as cl
from chainlit.types import AskFileResponse
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
class File:
    def __init__(self, path, type):
        self.path = path
        self.type = type

demo = File(path='./demo.pdf', type='application/pdf')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
embedding = HuggingFaceEmbeddings()

c:\Users\LONG\anaconda3\envs\rag-env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


2024-07-07 11:25:15 - Use pytorch device_name: cpu
2024-07-07 11:25:15 - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


c:\Users\LONG\anaconda3\envs\rag-env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
def process_file(file, text_splitter: RecursiveCharacterTextSplitter):
    """
    Processes the uploaded file by splitting it into smaller documents.

    Args:
        file (AskFileResponse): The uploaded file.
        text_splitter (RecursiveCharacterTextSplitter): The text splitter instance.

    Returns:
        List[Document]: A list of split documents with metadata.
    """
    if file.type == 'text/plain':
        loader = TextLoader(file.path)
    elif file.type == 'application/pdf':
        loader = PyPDFLoader(file.path)
    else:
        raise ValueError(f"Unsupported file type: {file.type}")

    documents = loader.load()
    docs = text_splitter.split_documents(documents)

    for i, doc in enumerate(docs):
        doc.metadata["source"] = f"source_{i}"

    return docs

In [24]:
docs = process_file(demo, text_splitter)
print(f"Number of documents: {len(docs)}")
print("First document:", docs[0])

Number of documents: 29
First document: page_content='AI VIET NAM – AI COURSE 2024
Kiểm tra tuân thủ đội mũ bảo vệ với YOLOv10
Dinh-Thang Duong, Quang-Vinh Dinh
Ngày 22 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh
vực Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng
trong một tấm ảnh. Đây là một trong những bài toán quan trọng và phức tạp trong Computer
Vision, với ứng dụng rộng rãi từ nhận diện khuôn mặt, nhận dạng biển số xe, đến theo dõi đối
tượng trong video và tự động lái xe.
Hình 1: Chương trình phát hiện đối tượng có mang mũ bảo hiểm.
Trong project này, chúng ta sẽ xây dựng một chương trình phát hiện các nhân viên có đeo mũ
bảo vệ trong công trường hay không? Mô hình mà chúng ta sử dụng sẽ là mô hình YOLOv10.
Theo đó, Input và Output của chương trình là:
•Input:Một tấm ảnh.
•Output: Tọa độ (bounding box) của các nhân viên và phần mũ bảo hiểm.' metadata={'source': 'source_

In [26]:
def get_vector_db(file, text_splitter, embedding):
    """
    Creates a vector database from the processed file.

    Args:
        file (AskFileResponse): The uploaded file.

    Returns:
        Chroma: The vector database instance.
    """
    docs = process_file(file, text_splitter)

    vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
    return vector_db

In [27]:
vector_db = get_vector_db(demo, text_splitter, embedding)
print(vector_db)

2024-07-07 11:39:56 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
